# doc2vecを用いた単語の特徴ベクトル学習

In [1]:
import gensim
import os
import collections
import random
import csv
import sys

# 準備

CSVの行サイズ上限を限界まで引き上げる

In [2]:
csv.field_size_limit(sys.maxsize)

131072

In [3]:
corpusfile = 'novel2vec.txt'
modelfile = 'novel2vec.model'

ファイルから小説データを読み取る。
学習データは94380件のpixiv小説。(2.1GB)

In [4]:
def read_corpus():
    with open(corpusfile, newline='', encoding='utf-8') as csvfile:
        novels = csv.reader(csvfile)
        for novel in novels:
            novel_id, tokens_text = novel
            tokens = gensim.utils.to_unicode(tokens_text).split()
            yield gensim.models.doc2vec.TaggedDocument(tokens, [novel_id])

In [5]:
corpus = list(read_corpus())

# 学習

モデルデータを構築

In [6]:
model = gensim.models.doc2vec.Doc2Vec(size=100, min_count=5, iter=20, workers=36)

In [7]:
model.build_vocab(corpus)

In [8]:
import resource
resource.getrusage(resource.RUSAGE_SELF).ru_maxrss

32803272

学習する

In [ ]:
%time model.train(corpus)

In [10]:
import resource
resource.getrusage(resource.RUSAGE_SELF).ru_maxrss

32875396

ファイルとしてモデルを保存

In [11]:
model.save(modelfile)

# 評価

学習データのうちの1000件の小説について、小説データから表現ベクトルを再学習し、学習データ中のどのデータに最も近いかを計算し、自分自身が何番目に来るかで評価する。

In [17]:
ranks = []
second_ranks = []
for doc_id in range(1000):
    inferred_vector = model.infer_vector(corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(corpus[doc_id].tags[0])
    ranks.append(rank)
    second_ranks.append(sims[1])

結果、認識率99.6%。

In [18]:
collections.Counter(ranks)

Counter({0: 996, 1: 1, 13: 1, 20: 1, 36: 1})